#  第12张 pandas高级应用 Advanced pandas

* 前⾯的章节关注于不同类型的数据规整流程和NumPy、 pandas
与其它库的特点。随着时间的发展， pandas发展出了更多适合
⾼级⽤户的功能。本章就要深⼊学习pandas的⾼级功能。

In [1]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.set_printoptions(precision=4, suppress=True)

##  12.1  分类数据 Categorical Data

* 这⼀节介绍的是pandas的分类类型。我会向你展示通过使⽤
它，提⾼性能和内存的使⽤率。我还会介绍⼀些在统计和机器学
习中使⽤分类数据的⼯具。

###  背景和目的 Background and Motivation

* 表中的⼀列通常会有重复的包含不同值的⼩集合的情况。我们已
经学过了unique和value_counts，它们可以从数组提取出不同的
值，并分别计算频率

In [3]:
import numpy as np; import pandas as pd
values = pd.Series(['apple', 'orange', 'apple',
                    'apple'] * 2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [5]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [6]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

* 许多数据系统（数据仓库、统计计算或其它应⽤）都发展出了特
定的表征重复值的⽅法，以进⾏⾼效的存储和计算。在数据仓库
中，最好的⽅法是使⽤所谓的包含不同值得维表(Dimension
Table)，将主要的参数存储为引⽤维表整数键：

In [10]:
values = pd.Series([0, 1, 0, 0] * 2)
dim = pd.Series(['apple', 'orange'])
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [11]:
dim

0     apple
1    orange
dtype: object

* 可以使⽤take⽅法存储原始的字符串Series：

In [12]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

这种⽤整数表示的⽅法称为分类或字典编码表示法。不同值得数
组称为分类、字典或数据级。本书中，我们使⽤分类的说法。表
示分类的整数值称为分类编码或简单地称为编码。

分类表示可以在进⾏分析时⼤⼤的提⾼性能。你也可以在保持编
码不变的情况下，对分类进⾏转换。⼀些相对简单的转变例⼦包
括：

* 重命名分类。
* 加⼊⼀个新的分类，不改变已经存在的分类的顺序或位置

###  pandas的分类类型 Categorical Type in pandas

* pandas有⼀个特殊的分类类型，⽤于保存使⽤整数分类表示法
的数据。看⼀个之前的Series例⼦

In [14]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])
df

,basket_id,fruit,count,weight
0,0,apple,9,0.201952
1,1,orange,10,3.224939
2,2,apple,14,3.723263
3,3,apple,10,1.456118
4,4,apple,10,2.763792
5,5,orange,11,0.517258
6,6,apple,10,3.330746
7,7,apple,4,1.273415


* 这⾥， df['fruit']是⼀个Python字符串对象的数组。我们可以通过
调⽤它，将它转变为分类：

In [15]:
fruit_cat = df['fruit'].astype('category')
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

* fruit_cat的值不是NumPy数组，⽽是⼀个pandas.Categorical实
例：

In [16]:
c = fruit_cat.values
type(c)

pandas.core.arrays.categorical.Categorical

* 分类对象有categories和codes属性：

In [17]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [18]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

* 你可将DataFrame的列通过分配转换结果，转换为分类：

In [19]:
df['fruit'] = df['fruit'].astype('category')
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

* 你还可以从其它Python序列直接创建pandas.Categorical：

In [21]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

* 如果你已经从其它源获得了分类编码，你还可以使⽤from_codes
构造器：

In [22]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

* 与显示指定不同，分类变换不认定指定的分类顺序。因此取决于
输⼊数据的顺序， categories数组的顺序会不同。当使⽤
from_codes或其它的构造器时，你可以指定分类⼀个有意义的顺
序：

In [23]:
ordered_cat = pd.Categorical.from_codes(codes, categories,
                                        ordered=True)
ordered_cat

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

* 输出[foo < bar < baz]指明‘foo’位于‘bar’的前⾯，以此类推。⽆序
的分类实例可以通过as_ordered排序：

In [24]:
my_cats_2.as_ordered()

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

最后要注意，分类数据不需要字符串，尽管我仅仅展示了字符串
的例⼦。分类数组可以包括任意不可变类型。

### ⽤分类进⾏计算 Computations with Categoricals

* 与⾮编码版本（⽐如字符串数组）相⽐，使⽤pandas的
Categorical有些类似。某些pandas组件，⽐如groupby函数，更
适合进⾏分类。还有⼀些函数可以使⽤有序标志位。
来看⼀些随机的数值数据，使⽤pandas.qcut⾯元函数。它会返
回pandas.Categorical，我们之前使⽤过pandas.cut，但没解释
分类是如何⼯作的：

In [25]:
np.random.seed(12345)
draws = np.random.randn(1000)
draws[:5]

array([-0.2047,  0.4789, -0.5194, -0.5557,  1.9658])

* 计算这个数据的分位⾯元，提取⼀些统计信息

In [26]:
bins = pd.qcut(draws, 4)
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

* 虽然有⽤，确切的样本分位数与分位的名称相⽐，不利于⽣成汇
总。我们可以使⽤labels参数qcut，实现⽬的：

In [28]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [29]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

* 加上标签的⾯元分类不包含数据⾯元边界的信息，因此可以使⽤
groupby提取⼀些汇总信息：

In [30]:
bins = pd.Series(bins, name='quartile')
results = (pd.Series(draws)
           .groupby(bins)
           .agg(['count', 'min', 'max'])
           .reset_index())
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


* 分位数列保存了原始的⾯元分类信息，包括排序：

In [31]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

#### 用分类提高性能  Better performance with categoricals

* 如果你是在⼀个特定数据集上做⼤量分析，将其转换为分类可以
极⼤地提⾼效率。 DataFrame列的分类使⽤的内存通常少的多。
来看⼀些包含⼀千万元素的Series，和⼀些不同的分类：

In [32]:
N = 10000000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

* 现在，将标签转换为分类：

In [34]:
categories = labels.astype('category')

* 这时，可以看到标签使⽤的内存远⽐分类多：

In [36]:
labels.memory_usage()

80000080

In [37]:
categories.memory_usage()

10000272

* 转换为分类不是没有代价的，但这是⼀次性的代价：

In [38]:
%time _ = labels.astype('category')

Wall time: 370 ms


* GroupBy操作明显⽐分类快，是因为底层的算法使⽤整数编码数
组，⽽不是字符串数组。

### 分类方法  Categorical Methods

* 包含分类数据的Series有⼀些特殊的⽅法，类似于Series.str字符
串⽅法。它还提供了⽅便的分类和编码的使⽤⽅法。看下⾯的
Series：

In [41]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

* 特别的cat属性提供了分类⽅法的⼊⼝：

In [44]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [45]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

* 假设我们知道这个数据的实际分类集，超出了数据中的四个值。
我们可以使⽤set_categories⽅法改变它们：

In [46]:
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

* 虽然数据看起来没变，新的分类将反映在它们的操作中。例如，
如果有的话， value_counts表示分类：

In [47]:
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [48]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

* 在打数据集中，分类经常作为节省内存和⾼性能的便捷⼯具。过
滤完⼤DataFrame或Series之后，许多分类可能不会出现在数据
中。我们可以使⽤remove_unused_categories⽅法删除没看到
的分类：

In [50]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [51]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

####   为建模创建虚拟变量  Creating dummy variables for modeling

* 当你使⽤统计或机器学习⼯具时，通常会将分类数据转换为虚拟
变量，也称为one-hot编码。这包括创建⼀个不同类别的列的
DataFrame；这些列包含给定分类的1s，其它为0。
看前⾯的例⼦：

In [52]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

* 前⾯的第7章提到过， pandas.get_dummies函数可以转换这个以
为分类数据为包含虚拟变量的DataFrame：

In [53]:
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


##  12.2 GroupBy⾼级应⽤ Advanced GroupBy Use

### 分组转换和“解封”GroupBy  Group Transforms and "Unwrapped" GroupBys

在第10章，我们在分组操作中学习了apply⽅法，进⾏转换。还
有另⼀个transform⽅法，它与apply很像，但是对使⽤的函数有
⼀定限制：

* 它可以产⽣向分组形状⼴播标量值
* 它可以产⽣⼀个和输⼊组形状相同的对象
* 它不能修改输⼊

In [54]:
# 举例如下：
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


* 按键进行分组

In [55]:
g = df.groupby('key').value
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

* 假设我们想产⽣⼀个和df['value']形状相同的Series，但值替换为
按键分组的平均值。我们可以传递函数lambda x: x.mean()进⾏
转换：

In [56]:
g.transform(lambda x: x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

* 对于内置的聚合函数，我们可以传递⼀个字符串假名作为
GroupBy的agg⽅法：

In [57]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

* 与apply类似， transform的函数会返回Series，但是结果必须与
输⼊⼤⼩相同。举个例⼦，我们可以⽤lambda函数将每个分组
乘以2：

In [58]:
g.transform(lambda x: x * 2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

*  再举⼀个复杂的例⼦，我们可以计算每个分组的降序排名：

In [59]:
g.transform(lambda x: x.rank(ascending=False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

* 看⼀个由简单聚合构造的的分组转换函数：

In [60]:
def normalize(x):
    return (x - x.mean()) / x.std()

* 我们⽤transform或apply可以获得等价的结果：

In [63]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [64]:
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

* 内置的聚合函数，⽐如mean或sum，通常⽐apply函数快，也⽐
transform快。这允许我们进⾏⼀个所谓的解封（unwrapped）分
组操作：

In [65]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [66]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

* 解封分组操作可能包括多个分组聚合，但是⽮量化操作还是会带
来收益。

### 分组的时间重采样 Grouped Time Resampling

* 对于时间序列数据， resample⽅法从语义上是⼀个基于内在时间
的分组操作。下⾯是⼀个示例表：

In [67]:
N = 15
times = pd.date_range('2017-05-20 00:00', freq='1min', periods=N)
df = pd.DataFrame({'time': times,
                   'value': np.arange(N)})
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


* 这⾥，我们可以⽤time作为索引，然后重采样：

In [68]:
df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


* 假设DataFrame包含多个时间序列，⽤⼀个额外的分组键的列进
⾏标记：

In [69]:
df2 = pd.DataFrame({'time': times.repeat(3),
                    'key': np.tile(['a', 'b', 'c'], N),
                    'value': np.arange(N * 3.)})
df2[:7]

,key,time,value
0,a,2017-05-20 00:00:00,0.0
1,b,2017-05-20 00:00:00,1.0
2,c,2017-05-20 00:00:00,2.0
3,a,2017-05-20 00:01:00,3.0
4,b,2017-05-20 00:01:00,4.0
5,c,2017-05-20 00:01:00,5.0
6,a,2017-05-20 00:02:00,6.0


* 要对每个key值进⾏相同的重采样，我们引⼊
pandas.TimeGrouper对象：

In [70]:
time_key = pd.TimeGrouper('5min')

c:\users\xiaomi\anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  if __name__ == '__main__':


* 我们然后设定时间索引，⽤key和time_key分组，然后聚合：

In [71]:
resampled = (df2.set_index('time')
             .groupby(['key', time_key])
             .sum())
resampled
resampled.reset_index()

,key,time,value
0,a,2017-05-20 00:00:00,30.0
1,a,2017-05-20 00:05:00,105.0
2,a,2017-05-20 00:10:00,180.0
3,b,2017-05-20 00:00:00,35.0
4,b,2017-05-20 00:05:00,110.0
5,b,2017-05-20 00:10:00,185.0
6,c,2017-05-20 00:00:00,40.0
7,c,2017-05-20 00:05:00,115.0
8,c,2017-05-20 00:10:00,190.0


* 使⽤TimeGrouper的限制是时间必须是Series或DataFrame的索
引。

## 12.3 链式编程技术Techniques for Method Chaining

* 当对数据集进⾏⼀系列变换时，你可能发现创建的多个临时变量
其实并没有在分析中⽤到。看下⾯的例⼦：

* 虽然这⾥没有使⽤真实的数据，这个例⼦却指出了⼀些新⽅法。
⾸先， DataFrame.assign⽅法是⼀个df[k] = v形式的函数式的列
分配⽅法。它不是就地修改对象，⽽是返回新的修改过的
DataFrame。因此，下⾯的语句是等价的

In [ ]:
# Usual non-functional way
df2 = df.copy()
df2['k'] = v

# Functional assign way
df2 = df.assign(k=v)

就地分配可能会⽐assign快，但是assign可以⽅便地进⾏链式编
程：

```python
result = (df2.assign(col1_demeaned=df2.col1 - df2.col2.mean())
          .groupby('key')
          .col1_demeaned.std())
```

我使⽤外括号，这样便于添加换⾏符。
使⽤链式编程时要注意，你可能会需要涉及临时对象。在前⾯的
例⼦中，我们不能使⽤load_data的结果，直到它被赋值给临时
变量df。为了这么做， assign和许多其它pandas函数可以接收类
似函数的参数，即可调⽤对象（callable）。为了展示可调⽤对
象，看⼀个前⾯例⼦的⽚段：

```python
df = load_data()
df2 = df[df['col2'] < 0]
```

它可以重写为

```python
df = (load_data()
      [lambda x: x['col2'] < 0])
```

这⾥， load_data的结果没有赋值给某个变量，因此传递到[ ]的函
数在这⼀步被绑定到了对象。
我们可以把整个过程写为⼀个单链表达式：

```python
result = (load_data()
          [lambda x: x.col2 < 0]
          .assign(col1_demeaned=lambda x: x.col1 - x.col1.mean())
          .groupby('key')
          .col1_demeaned.std())
```

是否将代码写成这种形式只是习惯⽽已，将它分开成若⼲步可以可读性

###  管道方法 The pipe Method

你可以⽤Python内置的pandas函数和⽅法，⽤带有可调⽤对象
的链式编程做许多⼯作。但是，有时你需要使⽤⾃⼰的函数，或
是第三⽅库的函数。这时就要⽤到管道⽅法。
看下⾯的函数调⽤：

```python
a = f(df, arg1=v1)
b = g(a, v2, arg3=v3)
c = h(b, arg4=v4)
```

当使⽤接收、返回Series或DataFrame对象的函数式，你可以调
⽤pipe将其重写：

```python
result = (df.pipe(f, arg1=v1)
          .pipe(g, v2, arg3=v3)
          .pipe(h, arg4=v4))
```

f(df)和df.pipe(f)是等价的，但是pipe使得链式声明更容易。
pipe的另⼀个有⽤的地⽅是提炼操作为可复⽤的函数。看⼀个从
列减去分组⽅法的例⼦：

```python
g = df.groupby(['key1', 'key2'])
df['col1'] = df['col1'] - g.transform('mean')
```

假设你想转换多列，并修改分组的键。另外，你想⽤链式编程做
这个转换。下⾯就是⼀个⽅法：

```python
def group_demean(df, by, cols):
    result = df.copy()
    g = df.groupby(by)
    for c in cols:
        result[c] = df[c] - g[c].transform('mean')
    return result
```

然后可以写为：

```python
result = (df[df.col1 < 0]
          .pipe(group_demean, ['key1', 'key2'], ['col1']))
```

In [76]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS

## 12.4 总结 Conclusion

和其它许多开源项⽬⼀样， pandas仍然在不断的变化和进步
中。和本书中其它地⽅⼀样，这⾥的重点是放在接下来⼏年不会
发⽣什么改变且稳定的功能。
为了深⼊学习pandas的知识，我建议你学习官⽅⽂档，并阅读
开发团队发布的更新⽂档。我们还邀请你加⼊pandas的开发⼯
作：修改bug、创建新功能、完善⽂档。